# Starboard News — Partner Integration Guide

This notebook demonstrates how a partner would integrate the Starboard News API into their website or app.

**Base URL:** `https://news.starboard.to`

**Authentication:** Public endpoints require no auth. Private endpoints require `Bearer CRON_SECRET`.

In [5]:
import requests
import json
from datetime import date

BASE_URL = "https://news.starboard.to"

def pp(resp):
    """Pretty print API response"""
    print(f"HTTP {resp.status_code}")
    try:
        print(json.dumps(resp.json(), indent=2, ensure_ascii=False))
    except:
        print(resp.text[:500])

---
## Step 1: Discover Available Newsletters

First, fetch the list of newsletters to display on your website.
This is public — no auth needed.

In [19]:
resp = requests.get(f"{BASE_URL}/api/newsletters")
pp(resp)

# Parse newsletter options for use in a UI
newsletters = resp.json()["newsletters"]
print(f"\nAvailable newsletters: {len(newsletters)}")
for n in newsletters:
    print(f"  {n['emoji']} {n['id']:20s} {n['name']} ({n['language']})")

HTTP 200
{
  "newsletters": [
    {
      "id": "minor_news",
      "name": "Minor News",
      "description": "Daily crypto & energy infrastructure news digest",
      "emoji": "⚡",
      "language": "en"
    },
    {
      "id": "into_crypto_cn",
      "name": "Into Crypto 中文版",
      "description": "每日加密货币深度分析",
      "emoji": "🪙",
      "language": "zh"
    },
    {
      "id": "into_crypto_en",
      "name": "Into Crypto",
      "description": "Daily crypto analysis and insights",
      "emoji": "🪙",
      "language": "en"
    }
  ]
}

Available newsletters: 3
  ⚡ minor_news           Minor News (en)
  🪙 into_crypto_cn       Into Crypto 中文版 (zh)
  🪙 into_crypto_en       Into Crypto (en)


---
## Step 2: Subscribe a User

When a user fills in their email and selects newsletters on your site,
make this POST request. The user will immediately receive the latest issue.

In [22]:
# Subscribe a user to two newsletters
resp = requests.post(f"{BASE_URL}/api/subscribe", json={
    "email": "zianliu05@gmail.com",
    "feeds": ["minor_news", "into_crypto_en", "into_crypto_cn"]
})
pp(resp)

HTTP 200
{
  "success": true,
  "message": "Subscription updated! Check your inbox.",
  "feeds": [
    "minor_news",
    "into_crypto_en",
    "into_crypto_cn"
  ]
}


In [ ]:
# Re-subscribing the same user with a new feed — merges, doesn't duplicate
resp = requests.post(f"{BASE_URL}/api/subscribe", json={
    "email": "partner-test@example.com",
    "feeds": ["into_crypto_cn"]
})
pp(resp)
# Notice: feeds now includes all 3

### Error Handling

Here's what your frontend should handle:

In [ ]:
# Invalid email
resp = requests.post(f"{BASE_URL}/api/subscribe", json={
    "email": "not-an-email",
    "feeds": ["minor_news"]
})
print("Invalid email ->")
pp(resp)
assert resp.status_code == 400

In [ ]:
# Invalid feed ID
resp = requests.post(f"{BASE_URL}/api/subscribe", json={
    "email": "test@example.com",
    "feeds": ["nonexistent_feed"]
})
print("Invalid feed ->")
pp(resp)
assert resp.status_code == 400

In [ ]:
# Empty feeds array
resp = requests.post(f"{BASE_URL}/api/subscribe", json={
    "email": "test@example.com",
    "feeds": []
})
print("Empty feeds ->")
pp(resp)
assert resp.status_code == 400

---
## Step 3: Fetch Today's Content

Partners can pull today's newsletter content to display on their own site.
Returns both raw markdown and pre-rendered HTML.

In [21]:
# Fetch Into Crypto CN content
resp = requests.get(f"{BASE_URL}/api/content/today?feed=into_crypto_cn")
pp(resp)

HTTP 200
{
  "date": "2026-02-11",
  "title": "Into Crypto 中文版",
  "emoji": "🪙",
  "content": "🪙 Into Crypto 中文版 | 2026-02-11\n\n比特币跌至6.9万美元附近，市场情绪谨慎；白宫今日召开稳定币高峰会议，银行与加密巨头同台博弈CLARITY法案。\n\n───\n\n1️⃣ 白宫召开第二次稳定币峰会，银行与加密巨头正面交锋\n\n白宫于2月10日召开第二次闭门会议，试图打破CLARITY法案（数字资产市场清晰法案）的立法僵局。与会者包括摩根大通、美国银行、富国银行等传统金融巨头，以及Coinbase、Circle、Tether等加密公司高管。核心争议焦点是稳定币发行方是否应被允许向持有者支付收益。\n\n稳定币是一种加密货币，其价值与美元等法定货币1:1挂钩。比如你持有1个USDC，它始终约等于1美元。稳定币让人们可以在加密世界里使用\"数字美元\"，既有加密货币快速转账的优势，又避免了比特币那样的价格剧烈波动。\n\n\"收益\"是指你把钱存放在某处获得的回报。传统银行给储蓄账户支付利息，现在一些稳定币发行商也想这么做——把他们用储备金投资赚的钱分一部分给用户。但银行担心这会抢走他们的存款客户，因为加密收益可能比银行利息更高。\n\n如果CLARITY法案允许稳定币支付收益，可能导致数十亿美元从银行存款转移到加密稳定币。这将重塑传统金融与加密行业的竞争格局，也可能改变普通人存钱的方式。\n\n值得思考：\n• 如果稳定币可以提供比银行更高的收益，银行凭什么留住储户？\n• 监管者应该保护现有金融体系，还是允许更激烈的竞争让消费者受益？\n\n🔗 https://coinfomania.com/white-house-stablecoin-meeting-banks-crypto-2026/\n\n───\n\n2️⃣ 高盛披露23.6亿美元加密货币持仓，华尔街加速入场\n\n投资银行巨头高盛在最新监管文件中披露了超过23.6亿美元的加密货币敞口：比特币约11亿美元、以太坊约10亿美元、XRP约1.53亿美元、Solana约1.08亿美元。这一数字较上季度增长约15%。\n\n13F文件是美国证券交易委员会（SEC）要求大型投资机构每季度提交的

In [14]:
# Fetch Minor News content
resp = requests.get(f"{BASE_URL}/api/content/today?feed=minor_news")

if resp.status_code == 200:
    data = resp.json()
    print(f"Date: {data['date']}")
    print(f"Title: {data['emoji']} {data['title']}")
    print(f"Content preview: {data['content'][:200]}...")
    print(f"HTML available: {len(data['html'])} chars")
elif resp.status_code == 404:
    print("No content available for today yet.")
    print("Content is generated daily around 07:00 UTC.")
else:
    pp(resp)

Date: 2026-02-11
Title: ⚡ Minor News
Content preview: Test content for minor_news...
HTML available: 92 chars


In [9]:
# Invalid feed returns 400 with list of valid feeds
resp = requests.get(f"{BASE_URL}/api/content/today?feed=invalid")
print("Invalid feed ->")
pp(resp)
assert resp.status_code == 400

Invalid feed ->
HTTP 400
{
  "error": "Missing or invalid feed. Valid feeds: minor_news, into_crypto_cn, into_crypto_en"
}


---
## Step 4: Unsubscribe

Every email includes an unsubscribe link with a unique token.
Partners can also build a custom unsubscribe UI using the API.

In [ ]:
# Missing token
resp = requests.get(f"{BASE_URL}/api/unsubscribe")
print("No token ->")
pp(resp)
assert resp.status_code == 400

In [ ]:
# Invalid token
resp = requests.get(f"{BASE_URL}/api/unsubscribe?token=invalid-token")
print("Invalid token ->")
pp(resp)
assert resp.status_code == 404

In [ ]:
# To test real unsubscribe, paste a token from the subscriptions table:
# UNSUB_TOKEN = "paste-real-token-here"
# resp = requests.get(f"{BASE_URL}/api/unsubscribe?token={UNSUB_TOKEN}")
# pp(resp)

---
## Step 5: Full Integration Example

Here's how a partner would build a complete subscription + content display flow:

In [ ]:
def partner_integration_demo(user_email, selected_feeds):
    """Simulates a partner's backend integration."""
    print(f"=== Partner Integration Demo ===")
    print(f"User: {user_email}")
    print(f"Selected: {selected_feeds}")
    print()

    # 1. Subscribe the user
    resp = requests.post(f"{BASE_URL}/api/subscribe", json={
        "email": user_email,
        "feeds": selected_feeds
    })

    if resp.status_code == 200:
        result = resp.json()
        print(f"[OK] Subscribed to {result['feeds']}")
        print(f"     User will receive the latest issue immediately.")
    else:
        error = resp.json().get("error", "Unknown error")
        print(f"[FAIL] {error}")
        return

    print()

    # 2. Fetch content to display on partner site
    for feed in selected_feeds:
        resp = requests.get(f"{BASE_URL}/api/content/today?feed={feed}")
        if resp.status_code == 200:
            data = resp.json()
            print(f"[OK] {data['emoji']} {data['title']} ({data['date']})")
            print(f"     Content: {len(data['content'])} chars markdown")
            print(f"     HTML:    {len(data['html'])} chars (ready to embed)")
        elif resp.status_code == 404:
            print(f"[--] {feed}: No content today (generated at 07:00 UTC)")
        else:
            print(f"[FAIL] {feed}: HTTP {resp.status_code}")

    print()
    print("Done. User is now subscribed and will receive daily emails at 08:00 UTC.")


# Run the demo
partner_integration_demo(
    user_email="demo@partner-site.com",
    selected_feeds=["minor_news", "into_crypto_cn"]
)

---
## API Reference

| Endpoint | Method | Auth | Description |
|----------|--------|------|-------------|
| `/api/newsletters` | GET | No | List available newsletters |
| `/api/subscribe` | POST | No | Subscribe user (sends latest issue) |
| `/api/content/today?feed=` | GET | No | Get today's content (markdown + HTML) |
| `/api/unsubscribe?token=` | GET | No | Unsubscribe via token |

### Newsletter IDs

| ID | Name | Language |
|----|------|----------|
| `minor_news` | Minor News | English |
| `into_crypto_cn` | Into Crypto 中文版 | Chinese |
| `into_crypto_en` | Into Crypto | English |

### Notes

- **CORS** enabled on all public endpoints — safe to call from browser JS
- **Idempotent** — re-subscribing merges feeds, never duplicates
- **Welcome email** — new subscribers get the latest issue immediately
- **Content schedule** — generated ~07:00 UTC, emails sent ~08:00 UTC

In [10]:
import requests
import json

BASE_URL = "https://news.starboard.to"

resp = requests.get(f"{BASE_URL}/api/content/today?feed=minor_news")
print(f"HTTP {resp.status_code}")
if resp.status_code == 200:
    data = resp.json()
    print(f"Date: {data['date']}")
    print(f"Title: {data['emoji']} {data['title']}")
    print(f"Content: {data['content'][:150]}...")
else:
    print(json.dumps(resp.json(), indent=2, ensure_ascii=False))

HTTP 404
{
  "error": "No content available"
}


In [11]:
# Try to ingest again
CRON_SECRET = "C97491B9-12BE-4229-8005-E8C5C771792C"

resp = requests.post(f"{BASE_URL}/api/content/ingest",
    headers={"Authorization": f"Bearer {CRON_SECRET}"},
    json={
        "type": "daily",
        "date": "2026-02-11",
        "content": "Test content for minor_news"
    }
)
print("Ingest response:")
print(f"HTTP {resp.status_code}")
print(json.dumps(resp.json(), indent=2, ensure_ascii=False))

print()

# Now try to fetch
resp2 = requests.get(f"{BASE_URL}/api/content/today?feed=minor_news")
print("Fetch response:")
print(f"HTTP {resp2.status_code}")
print(json.dumps(resp2.json(), indent=2, ensure_ascii=False))

Ingest response:
HTTP 200
{
  "success": true,
  "type": "daily",
  "date": "2026-02-11",
  "content_length": 27
}

Fetch response:
HTTP 200
{
  "date": "2026-02-11",
  "title": "Minor News",
  "emoji": "⚡",
  "content": "Test content for minor_news",
  "html": "<p style=\"margin: 16px 0; line-height: 1.8; color: #475569;\">Test content for minor_news</p>",
  "generated_at": "2026-02-11T02:57:15.678901+00:00"
}
